# LoRA on GPT2 model

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling 
from peft import LoraConfig, get_peft_model, TaskType
from datasets import load_dataset

c:\Users\KIIT0001\Desktop\ML-DL\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## 1. Load tokenizer and model
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token ## for padding
model = AutoModelForCausalLM.from_pretrained(model_name)

In [6]:
## Apply LoRA to model
lora_config = LoraConfig (
    r=8,  # LoRA rank
    lora_alpha=16,
    target_modules=["c_attn"], # Target attention layers in GPT-2
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 294,912 || all params: 124,734,720 || trainable%: 0.2364


c:\Users\KIIT0001\Desktop\ML-DL\venv\Lib\site-packages\peft\tuners\lora\layer.py:1803: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [ ]:
#3. dataset (Toy)
data = [
    {"text": "Question: What is the capital of India?\nAnswer: New Delhi"},
    {"text": "Question: Who wrote Romeo and Juliet?\nAnswer: William Shakespeare"},
    {"text": "Question: What's the boiling point of water?\nAnswer: 100°C"},
]

from datasets import Dataset
dataset = Dataset.from_list(data)

def tokenize_function(ex):
    return tokenizer(ex["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)


Map: 100%|██████████| 3/3 [00:00<00:00, 161.68 examples/s]


In [8]:
tokenized_dataset

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 3
})

In [9]:
#4. Training setup
training_args = TrainingArguments (
    output_dir="./lora-gpt2",
    per_device_train_batch_size=2,
    num_train_epochs=5,
    logging_steps=1,
    save_steps=5,
    save_total_limit=2,
    fp16=True,
    logging_dir='./logs',
    report_to="none"
)

trainer = Trainer(
    model = model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False) 
)

C:\Users\KIIT0001\AppData\Local\Temp\ipykernel_20460\208409945.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [10]:
#5. Train
trainer.train()

c:\Users\KIIT0001\Desktop\ML-DL\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
1,3.426800
2,3.814000
3,4.365100
4,3.372900
5,3.543300
6,3.158400
7,3.550500
8,3.241100
9,3.589200
10,3.453500


c:\Users\KIIT0001\Desktop\ML-DL\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=10, training_loss=3.5514732122421266, metrics={'train_runtime': 12.0766, 'train_samples_per_second': 1.242, 'train_steps_per_second': 0.828, 'total_flos': 983242506240.0, 'train_loss': 3.5514732122421266, 'epoch': 5.0})

In [11]:
#6. save the model
model.save_pretrained(".lora-adapter")

# INFERENCE

In [ ]:
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = PeftModel.from_pretrained(base_model, ".lora-adapter")


In [20]:
prompt = "Question: Who wrote Romeo and Juliet?"

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(
    **inputs,
    max_new_tokens=50,
    temperature=0.7,         # controls randomness (lower = more conservative)
    top_k=50,                # only sample from top 50 words
    top_p=0.9,               # nucleus sampling (cumulative prob cutoff)
    repetition_penalty=1.5,  # discourage repeating same phrases
    do_sample=True           # use sampling instead of greedy decoding
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question: Who wrote Romeo and Juliet?
There are several reasons why it is possible to write a romantic novel. Firstly, there's the fact that Shakespeare knew how long he wanted his work done; secondly people don't like short stories in which characters talk about their lives but have no interest
